In [1]:
!pip install torch torchvision torchaudio pandas numpy scikit-learn matplotlib seaborn nltk
import pandas as pd
import numpy as np
import re
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords

# GPU Kontrolü
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılan cihaz: {device}")

# Stopwords indirme
print("Stopwords indiriliyor...")
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Veriyi Yükle
print("Veri yükleniyor...")
imdb_data = pd.read_csv("IMDB Dataset.csv")
print(f"Toplam yorum: {len(imdb_data)}")

Kullanılan cihaz: cpu
Stopwords indiriliyor...


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Monster\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Veri yükleniyor...
Toplam yorum: 50000


In [2]:
# Temizleme Fonksiyonu
def clean_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

print("Veriler temizleniyor...")
imdb_data['cleaned_review'] = imdb_data['review'].apply(clean_text)
imdb_data['label'] = imdb_data['sentiment'].map({'positive': 1, 'negative': 0})

# Train/Test Split (random_state = 42)
X = imdb_data['cleaned_review']
y = imdb_data['label'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Eğitim Seti: {len(X_train)}")
print(f"Test Seti: {len(X_test)}")

Veriler temizleniyor...
Eğitim Seti: 40000
Test Seti: 10000


In [3]:
# SÖZLÜK VE TOKENIZATION FONKSİYONU
def build_vocab_and_sequences(texts, max_words=20000):
    word_freq = {}
    # Kelime Sayımı
    for text in texts:
        for word in text.split():
            if word not in stop_words:
                word_freq[word] = word_freq.get(word, 0) + 1
    
    # Sıralama ve En Sık Geçenleri Seçme
    # sorted_words'ü hesaplıyoruz ve return ile dışarı atıyoruz
    sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:max_words]
    
    vocab = {word: i+1 for i, (word, count) in enumerate(sorted_words)}
    vocab['<UNK>'] = len(vocab) + 1 # Bilinmeyen kelimeler için
    
    # Metinleri Sayı Dizisine Çevirme (Tokenization)
    sequences = []
    for text in texts:
        seq = [vocab.get(word, vocab['<UNK>']) for word in text.split() if word in vocab or word == '<UNK>']
        sequences.append(seq)
        
    # Fonksiyon artık sorted_words listesini de döndürüyor
    return vocab, sequences, sorted_words

print("Sözlük ve diziler oluşturuluyor...")

# Train seti üzerinden sözlük oluşturuluyor 
vocab, X_train_seq, sorted_words = build_vocab_and_sequences(X_train, max_words=20000)

# Test seti aynı sözlükle dönüştürülüyor
X_test_seq = []
for text in X_test:
    seq = [vocab.get(word, vocab['<UNK>']) for word in text.split() if word in vocab]
    X_test_seq.append(seq)

print(f"Vocabulary Boyutu: {len(vocab)}")
print(f"Stopwords Kümesi: {len(stop_words)} kelime")
print(f"\nEn Sık Kullanılan 10 Kelime:")
for word, freq in sorted_words[:10]:
    print(f"  {word}: {freq}")
print(f"\nEğitim Seti Dizi Sayısı: {len(X_train_seq)}")
print(f"Test Seti Dizi Sayısı: {len(X_test_seq)}")
print(f"\nÖrnek Yorum Metni: {X_train.iloc[0][:100]}")
print(f"Örnek Yorumun Sayı Hali: {X_train_seq[0][:15]}")

Sözlük ve diziler oluşturuluyor...
Vocabulary Boyutu: 20001
Stopwords Kümesi: 198 kelime

En Sık Kullanılan 10 Kelime:
  movie: 66900
  film: 59913
  one: 40389
  like: 30923
  good: 22828
  even: 19365
  would: 19238
  time: 18641
  really: 18383
  see: 17986

Eğitim Seti Dizi Sayısı: 40000
Test Seti Dizi Sayısı: 10000

Örnek Yorum Metni: i caught this little gem totally by accident back in or i was at a revival theatre to see two old si
Örnek Yorumun Sayı Hali: [896, 39, 1343, 328, 1478, 53, 8350, 1497, 10, 32, 75, 538, 763, 21, 1497]


In [4]:
# Padding
# Bu fonksiyon, farklı uzunluktaki yorumları alıp hepsini aynı boyut getirir. Eksik kısımları 0 ile doldurur.
def pad_sequences_manual(sequences, max_len=256):
    features = np.zeros((len(sequences), max_len), dtype=int)
    for i, seq in enumerate(sequences):
        if len(seq) != 0:
            features[i, -len(seq):] = np.array(seq)[:max_len]
    return features
    
# Fonksiyonu Eğitim ve Test verilerine uyguluyoruz
# Her yorumu kelime uzunluğuna sabitliyoruz.
X_train_padded = pad_sequences_manual(X_train_seq, max_len=256)
X_test_padded = pad_sequences_manual(X_test_seq, max_len=256)

print(f"Train verisi şekli: {X_train_padded.shape}")
print(f"Test verisi şekli: {X_test_padded.shape}")

Train verisi şekli: (40000, 256)
Test verisi şekli: (10000, 256)


In [5]:
# Eğitim ve test verilini Tensor'a çevir
train_data = TensorDataset(torch.from_numpy(X_train_padded), torch.from_numpy(y_train))
test_data = TensorDataset(torch.from_numpy(X_test_padded), torch.from_numpy(y_test))

# DataLoader
batch_size = 64 # Model her adımda 64 tane yorumu aynı anda okuyacak.
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

print(f"Batch Size: {batch_size}")
print(f"Train Loader Adım Sayısı: {len(train_loader)}")
print(f"Test Loader Adım Sayısı: {len(test_loader)}")

Batch Size: 64
Train Loader Adım Sayısı: 625
Test Loader Adım Sayısı: 157


In [6]:
# 1. Model: LSTM (Hafızalı Sinir Ağı)
class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentLSTM, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds)
        lstm_out = lstm_out[:, -1, :] 
        out = self.dropout(lstm_out)
        out = self.fc(out)
        return self.sigmoid(out)

# 2. Model: CNN (Metin İçin Konvolüsyonel Ağ)
class SentimentCNN(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim):
        super(SentimentCNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # Farklı boyutlarda filtreler (3'lü, 4'lü, 5'li kelime gruplarını yakalar)
        self.conv1 = nn.Conv1d(embedding_dim, 100, kernel_size=3)
        self.conv2 = nn.Conv1d(embedding_dim, 100, kernel_size=4)
        self.conv3 = nn.Conv1d(embedding_dim, 100, kernel_size=5)
        
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(300, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(0, 2, 1) # Boyut değişimi (Batch, Embed, Seq)
        
        x1 = self.pool(torch.relu(self.conv1(x))).squeeze(-1)
        x2 = self.pool(torch.relu(self.conv2(x))).squeeze(-1)
        x3 = self.pool(torch.relu(self.conv3(x))).squeeze(-1)
        
        x_concat = torch.cat((x1, x2, x3), dim=1)
        out = self.dropout(x_concat)
        out = self.fc(out)
        return self.sigmoid(out)
        
print("Modeller başarıyla tanımlandı.")

Modeller başarıyla tanımlandı.


In [7]:
# Eğitim Fonksiyonu 
def train_model(model, train_loader, epochs=1, lr=0.001):
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    
    print(f"--- {type(model).__name__} Eğitiliyor... ---")
    
    for epoch in range(epochs): 
        model.train()
        train_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device).long(), labels.to(device).float()
            model.zero_grad()
            output = model(inputs)
            loss = criterion(output.squeeze(), labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            
        print(f"Epoch {epoch+1} Tamamlandı. Loss: {train_loss/len(train_loader):.4f}")
            
    return model

In [8]:
# Tahmin Fonksiyonu
def get_predictions(model, loader):
    model.eval()
    all_preds = []
    with torch.no_grad():
        for inputs, _ in loader:
            inputs = inputs.to(device).long()
            outputs = model(inputs)
            preds = torch.round(outputs.squeeze()).cpu().numpy()
            all_preds.extend(preds)
    return np.array(all_preds)

In [9]:
vocab_size = len(vocab) + 2 # Pad ve Unk için

# LSTM Hiperparametreleri ve Eğitim
print("\n--- LSTM EĞİTİMİ ---")
model_lstm = SentimentLSTM(vocab_size, output_size=1, embedding_dim=100, hidden_dim=256, n_layers=2)
model_lstm = train_model(model_lstm, train_loader, epochs=3, lr=0.001)

# CNN Hiperparametreleri ve Eğitim
print("\n--- CNN EĞİTİMİ ---")
model_cnn = SentimentCNN(vocab_size, output_size=1, embedding_dim=100)
model_cnn = train_model(model_cnn, train_loader, epochs=3, lr=0.001)

print("\nEğitimler tamamlandı.")


--- LSTM EĞİTİMİ ---
--- SentimentLSTM Eğitiliyor... ---
Epoch 1 Tamamlandı. Loss: 0.6510
Epoch 2 Tamamlandı. Loss: 0.5732
Epoch 3 Tamamlandı. Loss: 0.3137

--- CNN EĞİTİMİ ---
--- SentimentCNN Eğitiliyor... ---
Epoch 1 Tamamlandı. Loss: 0.5694
Epoch 2 Tamamlandı. Loss: 0.4031
Epoch 3 Tamamlandı. Loss: 0.3245

Eğitimler tamamlandı.


In [14]:
# tahminleri al
y_pred_lstm = get_predictions(model_lstm, test_loader)
y_pred_cnn = get_predictions(model_cnn, test_loader)

def get_summary_metrics(y_true, y_pred, model_name):
    return {
        'Model': model_name,
        'Accuracy': accuracy_score(y_true, y_pred),
        'F1 Score (Weighted)': f1_score(y_true, y_pred, average='weighted'),
        'Precision (Weighted)': precision_score(y_true, y_pred, average='weighted'),
        'Recall (Weighted)': recall_score(y_true, y_pred, average='weighted')
    }

metrics_lstm = get_summary_metrics(y_test, y_pred_lstm, 'LSTM')
metrics_cnn = get_summary_metrics(y_test, y_pred_cnn, 'CNN')
# Önceki projeden alınan logistic regression model metrik değerleri
metrics_log = {'Model': 'Logistic Regression',
               'Accuracy': 0.8798,
               'F1 Score (Weighted)': 0.8808,
               'Precision (Weighted)': 0.8735,
               'Recall (Weighted)': 0.8882}

df_general = pd.DataFrame([metrics_lstm, metrics_cnn, metrics_log]).set_index('Model')

def highlight_max_gen(s):
    is_max = s == s.max()
    return ['font-weight: bold; background-color: #90ee90' if v else '' for v in is_max]

print("\n=== GENEL MODEL KARŞILAŞTIRMASI ===")
display(df_general.style.apply(highlight_max_gen))


=== GENEL MODEL KARŞILAŞTIRMASI ===


,Accuracy,F1 Score (Weighted),Precision (Weighted),Recall (Weighted)
Model,,,,
LSTM,0.879800,0.879799,0.879814,0.879800
CNN,0.874100,0.874035,0.874873,0.874100
Logistic Regression,0.879800,0.880800,0.873500,0.888200


In [15]:
report_lstm = classification_report(y_test, y_pred_lstm, output_dict=True)
report_cnn = classification_report(y_test, y_pred_cnn, output_dict=True)

# Önceki projeden alınan Lojistik Regresyon metrik değerleri
log_reg_data = {
    'Negative': {'p': 0.886289, 'r': 0.871400, 'f1': 0.878781},
    'Positive': {'p': 0.873524, 'r': 0.888200, 'f1': 0.880801}
}

class_metrics = []
classes = {'0': 'Negative', '1': 'Positive'}

for cls_key, cls_name in classes.items():
    
    class_metrics.append({'Sınıf': cls_name, 'Model': 'LSTM', 
                          'Precision': report_lstm[cls_key]['precision'], 
                          'Recall': report_lstm[cls_key]['recall'], 
                          'F1-Score': report_lstm[cls_key]['f1-score']})
    
    class_metrics.append({'Sınıf': cls_name, 'Model': 'CNN', 
                          'Precision': report_cnn[cls_key]['precision'], 
                          'Recall': report_cnn[cls_key]['recall'], 
                          'F1-Score': report_cnn[cls_key]['f1-score']})
    
    class_metrics.append({'Sınıf': cls_name, 'Model': 'Log Reg', 
                          'Precision': log_reg_data[cls_name]['p'], 
                          'Recall': log_reg_data[cls_name]['r'], 
                          'F1-Score': log_reg_data[cls_name]['f1']})

df_class_based = pd.DataFrame(class_metrics).set_index(['Sınıf', 'Model'])


def custom_style(df):
    styles = pd.DataFrame('', index=df.index, columns=df.columns)
    for idx in df.index:
        if idx[0] == 'Negative':
            styles.loc[idx, :] = 'background-color: #e0e0e0; color: black' # Gri
        else:
            styles.loc[idx, :] = 'background-color: white; color: black' # Beyaz

    # En Yüksek Değerleri Yeşil Yapma (Grup Bazlı)
    for col in df.columns:
        # Negative Grubu için Max
        max_neg = df.loc['Negative', col].max()
        # Positive Grubu için Max
        max_pos = df.loc['Positive', col].max()
        for idx in df.index:
            val = df.loc[idx, col]
            if idx[0] == 'Negative' and val == max_neg:
                styles.loc[idx, col] += '; background-color: #90ee90; font-weight: bold' # Açık Yeşil
            elif idx[0] == 'Positive' and val == max_pos:
                styles.loc[idx, col] += '; background-color: #90ee90; font-weight: bold' # Açık Yeşil      
    return styles

print("\n=== SINIF BAZLI BAŞARIM ===")
# style.apply(..., axis=None) tüm tabloyu fonksiyona gönderir
display(df_class_based.style.apply(custom_style, axis=None))

best_model = df_general['Accuracy'].idxmax()
best_acc = df_general['Accuracy'].max()

print("\n" + "="*50)
print(f"EN İYİ MODEL: {best_model} (%{best_acc*100:.2f})")


=== SINIF BAZLI BAŞARIM ===



EN İYİ MODEL: LSTM (%87.98)
